**Ultrasound Simulations Using the MRI-based and CT-based Skull Masks**
----
### Context
This notebook shows an example that takes a list of targets derived from EEG locations calculated with SimNIBS charm's processing.

In this demonstration, BabelBrain's (v0.3.5) domain generation and transcranial ultrasound modeling is executed for 54 targets.

This example can be modified to any other source of targets.

###  Dataset
We utilize the dataset with an SDR of 0.31, 0.42, 0.55, 0.67 and 0.79 which is accessible from the BabelBrain Zenodo repository. This code can also be adapted for different SDR values. Download it and keep note of the file path to it.

https://doi.org/10.5281/zenodo.7894431

### Create list of EEG locationss
In a separate terminal, we open a conda environment pointing to SimNIBS, for example:

conda activate /Applications/SimNIBS-4.1/simnibs_env/

Edit the file eeg_normals.py to indicate the paths to the SimNIBS output for the dataset, and run the script with

python eeg_normals.py



In [1]:
import numpy as np
import pandas as pd
import sys
import os
import time
from multiprocessing import Process, Queue
import multiprocessing
import nibabel
from pathlib import Path
from glob import glob
from scipy import ndimage
import shutil

## Additions to path
We add the paths to BabelBrain

In [ ]:
sys.path.append('BabelBrain_Path')     #Define the path to the BabelBrain folder
sys.path.append('BabelBrain_Path/BabelBrain')

In [ ]:
from BabelBrain.CalculateMaskProcess import CalculateMaskProcess
from TranscranialModeling.BabelIntegrationBASE import GetSmallestSOS
#we use the Single Element sim setting
from BabelBrain.CalculateFieldProcess import CalculateFieldProcess 
from BabelBrain.Babel_Thermal.CalculateThermalProcess import CalculateThermalProcess
from ThermalModeling.CalculateTemperatureEffects import GetThermalOutName

## Supportive functions
To read EEG poistions and its normals, and create Brainsight compatible trajectory definitions

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    if any(v): #if not all zeros then 
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return np.eye(3) #cross of all zeros only occurs on identical directions

In [ ]:
templateBSight=\
'''# Version: 13
# Coordinate system: NIfTI:Aligned
# Created by: Brainsight 2.5
# Units: millimetres, degrees, milliseconds, and microvolts
# Encoding: UTF-8
# Notes: Each column is delimited by a tab. Each value within a column is delimited by a semicolon.
# Target Name	Loc. X	Loc. Y	Loc. Z	m0n0	m0n1	m0n2	m1n0	m1n1	m1n2	m2n0	m2n1	m2n2
'''
lineBSight=\
'''{name}\t{X:10.9f}\t{Y:10.9f}\t{Z:10.9f}\t{m0n0:10.9f}\t{m0n1:10.9f}\t{m0n2:10.9f}\t{m1n0:10.9f}\t{m1n1:10.9f}\t{m1n2:10.9f}\t{m2n0:10.9f}\t{m2n1:10.9f}\t{m2n2:10.9f}
'''

**Locations**
----
Specific locations (Fp1, Fp2, Fpz, LPA, Nz, P9, P10, RPA, Afz, AF8, AF7, AF4, AF3) were omitted due to complications such as passage through air-filled structures, proximity to the ear, or issues with mask generation attributed to defacing. Consequently, 54 locations were suitable for analysis.

In [ ]:
## this file contains YES and NO for each EEG landmark to indicate if we want to include it
SELECTED_FOR_FUS = pd.read_csv('SELECTED_EEG.csv',index_col='Name')['Include'].to_dict()

In [ ]:
def ConvertEEGToBSightFUS(inputcsv='ernie_eeg_normals.csv',
                          outputBSight='ernie_bsight.txt',
                          depthFromSkin=0.0):
    MatResults={}
    A=pd.read_csv(inputcsv)
    with open(outputBSight,'w') as f:
        f.write(templateBSight)
        for index,entry in A.iterrows():
            transform=np.eye(4)
            NormalV=np.array(entry[['Nx','Ny','Nz']].tolist())
            transform[:3,:3]=rotation_matrix_from_vectors(np.array([0,0,-1]),NormalV)
            transform[:3,3]=np.array(entry[['R','A','S']].tolist())+depthFromSkin*NormalV
            outString=lineBSight.format(m0n0=transform[0,0],
                                    m0n1=transform[1,0],
                                    m0n2=transform[2,0],
                                    m1n0=transform[0,1],
                                    m1n1=transform[1,1],
                                    m1n2=transform[2,1],
                                    m2n0=transform[0,2],
                                    m2n1=transform[1,2],
                                    m2n2=transform[2,2],
                                    X=transform[0,3],
                                    Y=transform[1,3],
                                    Z=transform[2,3],
                                    name=entry['Name'])
            if SELECTED_FOR_FUS[entry['Name']]=='YES':
                f.write(outString)
                MatResults[entry['Name']]=transform
    return MatResults

def SaveMatFile(outfile,ID,transform):
    with open(outfile,'w') as f:
        f.write(templateBSight)
        outString=lineBSight.format(m0n0=transform[0,0],
                                m0n1=transform[1,0],
                                m0n2=transform[2,0],
                                m1n0=transform[0,1],
                                m1n1=transform[1,1],
                                m1n2=transform[2,1],
                                m2n0=transform[0,2],
                                m2n1=transform[1,2],
                                m2n2=transform[2,2],
                                X=transform[0,3],
                                Y=transform[1,3],
                                Z=transform[2,3],
                                name=ID)
        f.write(outString)
            

## Function to generate input files from trajectories
It calls the same code as in the BabelBrain's GUI step 1

In [ ]:
def RunMaskGeneration(T1W,
                      ID,
                      simbnibs_path,
                      Mat4Trajectory,
                      ComputingDevice='M1',
                      ComputingBackend=3, 
                      Frequency=500e3,
                      BasePPW=6,
                      TxSystem='Single',
                      bUseCT=True,
                      CT_or_ZTE_input='',
                      ZTERange=[0.1,0.65],
                      HUThreshold=300.0,
                      bReuseFiles=True,
                      CTType=1):
    

    print("*"*40)
    print("*"*5+" Calculating mask.. BE PATIENT... it can take a couple of minutes...")
    print("*"*40)

    deviceName=ComputingDevice
    COMPUTING_BACKEND=ComputingBackend

    T1WIso= T1W
   
    SmallestSoS= GetSmallestSOS(Frequency,bShear=True)

    prefix = ID + '_' + TxSystem +'_%ikHz_%iPPW_' %(int(Frequency/1e3),BasePPW)

    SpatialStep=np.round(SmallestSoS/Frequency/BasePPW*1e3,3) #step of mask to reconstruct , mm
    print("Frequency, SmallestSoS, BasePPW, SpatialStep",Frequency, SmallestSoS, BasePPW,SpatialStep)

    print("Mat4Trajectory",Mat4Trajectory)

    kargs={}
    kargs['SimbNIBSDir']=simbnibs_path
    kargs['SimbNIBSType']='charm'
    kargs['CoregCT_MRI']=True
    kargs['TrajectoryType']='brainsight'
    kargs['Mat4Trajectory']=Mat4Trajectory
    kargs['T1Source_nii']=T1W
    kargs['T1Conformal_nii']=T1WIso
    kargs['nIterationsAlign']=10
    kargs['SpatialStep']=SpatialStep
    kargs['InitialAligment']='HF'
    kargs['Location']=[0,0,0] #This coordinate will be ignored
    kargs['prefix']=prefix
    kargs['bPlot']=False
    kargs['bAlignToSkin']=True
    
    if bUseCT:
        kargs['CT_or_ZTE_input']=CT_or_ZTE_input
        kargs['CTType']=CTType
        if kargs['CTType'] in [2,3]:
            kargs['ZTERange']=ZTERange
        kargs['HUThreshold']=HUThreshold

    # Start mask generation as a separate process.
    queue=Queue()
    print(COMPUTING_BACKEND,deviceName,kargs)
    maskWorkerProcess = Process(target=CalculateMaskProcess, 
                                args=(queue,
                                     COMPUTING_BACKEND,
                                     deviceName),
                                kwargs=kargs)
    maskWorkerProcess.start()      
    # progress.
    T0=time.time()
    bNoError=True
    while maskWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False

    maskWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE calculating mask.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when generating mask!!")


## Function to run transcranial ultrasound modelling
It calls the same code as in the BabelBrain's GUI step 2

In [ ]:
def DistanceOutPlaneToFocus(FocalLength,Diameter):
    return np.sqrt(FocalLength**2-(Diameter/2)**2)
    
def RunAcousticSim(T1W,
                  ID,
                  TxSystem='Single',
                  deviceName='M1',
                  Aperture = 50e-3, #in m
                  FocalLength = 50e-3, #in m
                  Frequency=500e3, #in Hz
                  PPW=6,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=40e-3, #distance to run simulation beyond target
                  TxAdjustmentZ = 0.0, 
                  bUseCT = True):


    basedir,pID=os.path.split(os.path.split(T1W)[0])
    
    Target=[ID+'_'+TxSystem]

    InputSim=os.path.join(os.path.split(T1W)[0],ID+'_Single_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    print('InputSim',InputSim, basedir,pID)

    MaskData = nibabel.load(InputSim)
    FinalMask=np.flip(MaskData.get_fdata(),axis=2)

    VoxelSize=MaskData.header.get_zooms()[0]/1e3
    TargetLocation =np.array(np.where(FinalMask==5.0)).flatten()
    LineOfSight=FinalMask[TargetLocation[0],TargetLocation[1],:]
    StartSkin=np.where(LineOfSight>0)[0].min()
    DistanceFromSkin = (TargetLocation[2]-StartSkin)*VoxelSize
    

    Diameter = Aperture
    DOut=DistanceOutPlaneToFocus(FocalLength,Diameter)
    ZMax=DOut-DistanceFromSkin
    ZMaxSkin = np.round(ZMax*1e3,1)/1e3
    print('ZMaxSkin',ZMaxSkin,'DistanceFromSkin',DistanceFromSkin,'DOut',DOut)
    

    
    #no relocation of Tx
    TxMechanicalAdjustmentX= 0.0 #in m
    TxMechanicalAdjustmentY= 0.0 #in m
    TxMechanicalAdjustmentZ= ZMaxSkin+TxAdjustmentZ  #in m
    print('DistanceFromSkin',DistanceFromSkin,'ZMaxSkin',ZMaxSkin,'TxMechanicalAdjustmentZ',TxMechanicalAdjustmentZ)
    ZIntoSkin =0.0
    CurDistance=ZMaxSkin-TxMechanicalAdjustmentZ
    if CurDistance < 0:
        ZIntoSkin = np.abs(CurDistance)

    extrasuffix='Foc%03.1f_Diam%03.1f_' %(FocalLength*1e3,Aperture*1e3)
   
    Frequencies = [Frequency]
    basePPW=[PPW]
    T0=time.time()
    kargs={}
    kargs['extrasuffix']=extrasuffix
    kargs['ID']=pID
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['basePPW']=basePPW
    kargs['basedir']=basedir+os.sep
    kargs['Aperture']=Aperture
    kargs['FocalLength']=FocalLength
    kargs['TxMechanicalAdjustmentZ']=TxMechanicalAdjustmentZ
    kargs['TxMechanicalAdjustmentX']=TxMechanicalAdjustmentX
    kargs['TxMechanicalAdjustmentY']=TxMechanicalAdjustmentY
    kargs['ZIntoSkin']=ZIntoSkin
    kargs['Frequencies']=Frequencies
    kargs['zLengthBeyonFocalPointWhenNarrow']=zLengthBeyonFocalPointWhenNarrow
    kargs['bUseCT']=bUseCT
    kargs['bPETRA'] = False
    kargs['bMinimalSaving']=False
    kargs['bUseRayleighForWater']= True 

    # Start mask generation as separate process.
    bNoError=True
    queue=Queue()
    T0=time.time()
    fieldWorkerProcess = Process(target=CalculateFieldProcess, 
                                args=(queue,Target,TxSystem),
                                kwargs=kargs)
    fieldWorkerProcess.start()      
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE ultrasound simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running transcranial simulation!!")

# Run for SDR = All - T1W+T2W-derived mask only
Basic definitions of location, frequency, aperture and focal length

Instead of using the charm derived locations-orientations, we used a manually defined trajectory for the locations of: O1, O2 for SDR op67

### Convert all  EEG locations 
We add **30 mm** inside the brain from the EEG location (skin) as the LIFU target.

**Verify that EEG files file names match to what was done when running `0_eeg_normals.py` as indicated in the [README](README.md) file.

In [ ]:
eeg_files = [
    'SDR_0p31_eeg_normals.csv', 'SDR_0p42_eeg_normals.csv',
    'SDR_0p55_eeg_normals.csv', 'SDR_0p67_eeg_normals.csv',
    'SDR_0p79_eeg_normals.csv'
]

output_files = [
    'SDR_0p31_bsight_FUS_30mm.txt', 'SDR_0p42_bsight_FUS_30mm.txt',
    'SDR_0p55_bsight_FUS_30mm.txt', 'SDR_0p67_bsight_FUS_30mm.txt',
    'SDR_0p79_bsight_FUS_30mm.txt'
]


In [ ]:

TxSystem = 'Single'
Aperture = 50e-3
FocalLength = 50e-3
zLengthBeyonFocalPointWhenNarrow = 80e-3

ComputingDevice = 'M1'

root_path = 'The_path_to_the_participant_folder'

frequencies = [250e3, 500e3, 750e3]
PPWs = {250e3:9, 500e3:6, 750e3:6}

base_paths = [
    root_path + '/SDR_0p31/m2m_SDR_0p31', root_path + '/SDR_0p42/m2m_SDR_0p42',
    root_path + '/SDR_0p55/m2m_SDR_0p55', root_path + '/SDR_0p67/m2m_SDR_0p67',
    root_path + '/SDR_0p79/m2m_SDR_0p79'
]

t1w_files = [
    root_path + '/SDR_0p31/T1W.nii.gz', root_path + '/SDR_0p42/T1W.nii.gz',
    root_path + '/SDR_0p55/T1W.nii.gz', root_path + '/SDR_0p67/T1W.nii.gz',
    root_path + '/SDR_0p79/T1W.nii.gz'
]


In [ ]:
for Frequency in frequencies:
    for BasePath, T1W, output_txt in zip(base_paths, t1w_files, output_files):
        simbnibs_path = BasePath
        MatResults = ConvertEEGToBSightFUS(inputcsv=BasePath.split('/')[-2] + '_eeg_normals.csv', outputBSight=output_txt, depthFromSkin=30.0)
        for ID in MatResults:
            file_name = f'{ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_BabelViscoInput.nii.gz'
            full_path = os.path.join(Path(BasePath).parent, file_name)
            Mat4Trajectory = os.path.join(BasePath, ID+'.txt')
            ManualTrajectory = Mat4Trajectory.replace('.txt','_manual.txt')
            if os.path.isfile(ManualTrajectory): #we use the manually defined trajectory
                shutil.copyfile(ManualTrajectory,Mat4Trajectory)
            else:
                SaveMatFile(Mat4Trajectory, ID, MatResults[ID])
            if os.path.isfile(full_path):
                print('skipping', ID)
                continue
            RunMaskGeneration(T1W, 
                              ID, 
                              simbnibs_path, 
                              Mat4Trajectory, 
                              ComputingDevice=ComputingDevice, 
                              ComputingBackend=3, 
                              Frequency=Frequency, 
                              BasePPW=PPWs[Frequency], 
                              TxSystem=TxSystem, 
                              bUseCT=False, 
                              CT_or_ZTE_input='', 
                              ZTERange=[0.1, 0.65], 
                              HUThreshold=300.0, 
                              CTType=1)
            file_name = f'{ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_T1W_Resampled.nii.gz'
            T1w_resampled =  os.path.join(Path(BasePath).parent, file_name)
            #we delete this to save space
            os.remove(T1w_resampled)
            file_name = f'{ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW__box_FOV.stl'
            T1w_resampled =  os.path.join(Path(BasePath).parent, file_name)
            #we delete this to save space
            os.remove(T1w_resampled)



## Run all simulations

In [ ]:
for Frequency in frequencies:
    for BasePath, T1W in zip(base_paths, t1w_files):
        simbnibs_path = BasePath
        MatResults = ConvertEEGToBSightFUS(inputcsv=BasePath.split('/')[-2] + '_eeg_normals.csv', outputBSight=output_txt, depthFromSkin=30.0)
        for ID in MatResults:
            file_name = f'{ID}_Single_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}_DataForSim.h5'
            full_path = os.path.join(Path(BasePath).parent, file_name)
            if os.path.isfile(full_path):
                print('skipping', ID)
                continue
            RunAcousticSim(T1W,
                           ID,
                           TxSystem=TxSystem,
                           deviceName=ComputingDevice,
                           Aperture=Aperture,  # in m
                           FocalLength=FocalLength,  # in m
                           Frequency=Frequency,  # in Hz
                           PPW=PPWs[Frequency],
                           COMPUTING_BACKEND=3,
                           zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow,  # distance to run simulation beyond target
                           bUseCT=False)
            # we delete all files we won't use
            spath=os.path.join(Path(BasePath).parent,f'{ID}_Single_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}')+'*'
            ofiles=glob(spath)
            ofiles.sort()
            for f in ofiles:
                fn =  os.path.basename(f)
                if 'Water' in fn or '_FullElasticSolution.nii.gz' in fn or '.stl' in fn:
                    os.remove(f)

# Run for SDR = All - CT-T1W+T2W-derived


In [ ]:
ct_files = [
    root_path + '/SDR_0p31/CT.nii.gz', root_path + '/SDR_0p42/CT.nii.gz',
    root_path + '/SDR_0p55/CT.nii.gz', root_path + '/SDR_0p67/CT.nii.gz',
    root_path + '/SDR_0p79/CT.nii.gz'
]



In [ ]:
for Frequency in frequencies:
    for BasePath, T1W, CT in zip(base_paths, t1w_files, ct_files):
        simbnibs_path = BasePath
        MatResults = ConvertEEGToBSightFUS(inputcsv=BasePath.split('/')[-2] + '_eeg_normals.csv', outputBSight=output_txt, depthFromSkin=30.0)
        for ID in MatResults:
            CT_ID = 'CT_' + ID
            file_name = f'{CT_ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_BabelViscoInput.nii.gz'
            full_path = os.path.join(Path(BasePath).parent, file_name)
            Mat4Trajectory = os.path.join(BasePath, CT_ID+'.txt')
            ManualTrajectory = Mat4Trajectory.replace('.txt','_manual.txt')
            if os.path.isfile(ManualTrajectory): #we use the manually defined trajectory
                shutil.copyfile(ManualTrajectory,Mat4Trajectory)
            else:
                SaveMatFile(Mat4Trajectory, ID, MatResults[ID])
            if os.path.isfile(full_path):
                print('skipping', ID)
                continue
            RunMaskGeneration(T1W,
                              CT_ID,
                              simbnibs_path,
                              Mat4Trajectory,
                              ComputingDevice=ComputingDevice,
                              ComputingBackend=3,  # Metal
                              Frequency=Frequency,
                              BasePPW=PPWs[Frequency],
                              TxSystem=TxSystem,
                              bUseCT=True,
                              CT_or_ZTE_input=CT,
                              ZTERange=[0.1, 0.65],
                              HUThreshold=300.0,
                              CTType=1,
                              bReuseFiles=True)

            file_name = f'{CT_ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_T1W_Resampled.nii.gz'
            T1w_resampled =  os.path.join(Path(BasePath).parent, file_name)
            #we delete this to save space
            os.remove(T1w_resampled)

            
            if ID == 'F5' and 'SDR_0p31' in BasePath:
                print('Fixing mask')
                #this particular case has so much difference in the mask that scalp tissue is classified as brain tissue, we do a simple correction to reassign incorrect brain tissue back into scalp
                CT_Mask=nibabel.load(full_path)
                CT_mask_data=CT_Mask.get_fdata().astype(np.uint8)
                New_CT_mask_data=CT_mask_data.copy()
                ## we do a very simple operation to convert scalp voxels that are in contact to brain tissue back into scalp
                boneregion=(New_CT_mask_data==2) | (New_CT_mask_data==3)
                brainregion=New_CT_mask_data ==4
                iterations = {250e3:18,500e3:24,750e3:48}
                for n in range(iterations[Frequency]):
                    #we go by one iteration and use the skull region as a barrier to avoid bleeding into the skull cavity
                    if n==0:
                        Scalp_dilated= ndimage.binary_dilation(New_CT_mask_data==1,iterations=1)
                    else:
                        Scalp_dilated= ndimage.binary_dilation(Scalp_dilated,iterations=1)
                    Scalp_dilated[boneregion]=False
                    New_CT_mask_data[Scalp_dilated & brainregion]=1
                New_CT_mask = nibabel.Nifti1Image(New_CT_mask_data,CT_Mask.affine)
                New_CT_mask.to_filename(full_path)

## Run all simulations

In [ ]:
for Frequency in frequencies:
    for BasePath, T1W, CT in zip(base_paths, t1w_files, ct_files):
        
        simbnibs_path = BasePath
        MatResults = ConvertEEGToBSightFUS(inputcsv=BasePath.split('/')[-2] + '_eeg_normals.csv', outputBSight=output_txt, depthFromSkin=30.0)
        for ID in MatResults:
            CT_ID = 'CT_' + ID

            file_name = f'{CT_ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_BabelViscoInput.nii.gz'
            full_path = os.path.join(Path(BasePath).parent, file_name)
            if not os.path.isfile(full_path):
                print('skipping - missing BabelViscoInput', CT_ID)
                continue
            
            file_name = f'{CT_ID}_Single_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}_DataForSim.h5'
            full_path = os.path.join(Path(BasePath).parent, file_name)
            if os.path.isfile(full_path):
                print('skipping', CT_ID)
                continue

            RunAcousticSim(T1W,
                           CT_ID,
                           TxSystem=TxSystem,
                           deviceName=ComputingDevice,
                           Aperture=Aperture,  # in m
                           FocalLength=FocalLength,  # in m
                           Frequency=Frequency,  # in Hz
                           PPW=PPWs[Frequency],
                           COMPUTING_BACKEND=3,
                           zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow,  # distance to run simulation beyond target
                           bUseCT=True)
            spath=os.path.join(Path(BasePath).parent,f'{CT_ID}_Single_{int(Frequency/1e3)}kHz_{PPWs[Frequency]}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}')+'*'
            ofiles=glob(spath)
            ofiles.sort()
            for f in ofiles:
                fn =  os.path.basename(f)
                if 'Water' in fn or '_FullElasticSolution.nii.gz' in fn or '.stl' in fn:
                    os.remove(f)
    